In [ ]:
from pandas import DataFrame, Series, concat

from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, CARTODBPOSITRON
from bokeh.io import output_notebook, show
from bokeh.models import LogColorMapper, ColumnDataSource
from bokeh.palettes import Oranges256 as oranges

from pyproj import Proj, transform

from sparql_dataframe import get
endpoint = 'https://dati.cultura.gov.it/sparql'

In [ ]:
query = '''
SELECT DISTINCT(?institute) COUNT(DISTINCT(?s) as ?count)  ?lat ?long WHERE {
?s a ?o.
FILTER (?o = arco:CulturalProperty || ?o = arco:IntangibleCulturalProperty || ?o = arco:TangibleCulturalProperty || ?o = arco:ArchaeologicalProperty || ?o = arco:ImmovableCulturalProperty || ?o = arco:ArchitecturalOrLandscapeHeritage || ?o = arco:HistoricOrArtisticProperty || ?o = arco:MusicHeritage || ?o = arco:NaturalHeritage || ?o = arco:BotanicalHeritage || ?o = arco:MineralHeritage || ?o = arco:PalaeontologicalHeritage || ?o = arco:PertologicHeritage || ?o = arco:PlanetaryScienceHeritage || ?o = arco:ZoologicalHeritage || ?o = arco:NumismaticProperty || ?o = arco:PhotographicHeritage || ?o = arco:ScientificOrTechnologicalHeritage)
?s a-loc:hasCulturalInstituteOrSite ?institute.
  ?s clvapit:hasGeometry ?geometry .
  ?geometry a-loc:hasCoordinates ?coordinates .
?coordinates a-loc:lat ?lat.
?coordinates a-loc:long ?long.
}
'''
CP_coordinates = get(endpoint, query)

In [ ]:
query = '''
SELECT DISTINCT(?urlcity) ?dbpedia WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity owl:sameAs ?urlcity2.
?urlcity2 owl:sameAs ?dbpedia.
FILTER (contains(str(?dbpedia), "dbpedia"))
}
'''

dbPediaLinks = get(endpoint, query)

dbpedia = dbPediaLinks["dbpedia"]

link = []

CE_coordinates = DataFrame({"latitude":[], "longitude":[]})

for idx, item in dbpedia.items():
    query = '''
    SELECT ?latitude ?longitude WHERE {
       '''f'<{item}>''''  geo:lat ?latitude;
        geo:long ?longitude.
    } '''
    coordinates = get(endpoint, query)
    link.append(item)
    CE_coordinates = concat([CE_coordinates, coordinates])

CE_coordinates = CE_coordinates.insert(0, "dbPedia", link)